In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
targetAttribute = 'Poisonous/Edible'
minimalGainThreshold = 1e-6
#targetAttribute = ''
targetValue = []

In [3]:
# data class to hold csv data
def entropy(df):
    #target unique value calculation in form of dictionary 
    targetDict = pd.value_counts(df[targetAttribute]).to_dict()
    dfLength =len(df)
    
    totalEntropy = 0
    #print(targetDict)
    for key, val in targetDict.items():
        multipleVal = val/dfLength
        totalEntropy += multipleVal * math.log2(multipleVal)
    
    return 0-totalEntropy

In [4]:
#add values to targetValue
def initializeList(data):
    groupedData = data.groupby([targetAttribute]).groups.keys()
    return groupedData
    

In [5]:
def calcInforGainForEachAtrribute(df):
    # return maximum info gain
    maxInfoGain =0;
    infoGainDict = {}
    attributes = list(df)
    totalEntropy = entropy(df)

    for att in attributes:
        #don't calculate gain for target attribute
        if (att ==targetAttribute ):
            continue
        grouped_data = df.groupby([att], as_index=False)
        totalRows = len(df)
        subEntropy = 0
        #print('keys for ', att, ' total- ', totalRows)
        
        for key, value in grouped_data:
            eachGroupRows = len(value)
            #print('key-', key, ' eachGroupRows- ', eachGroupRows)
            
            S = entropy(value)
            
            #calculate |Sv/S|
            valEntrpy = eachGroupRows / totalRows
            
            #calculate |Sv/S|*Entropy(Sv)
            subEntropy = subEntropy + valEntrpy * S

        individualEntrpyGain = totalEntropy - subEntropy
        infoGainDict[att] = individualEntrpyGain
        #print('gain-', att, ' - ', individualEntrpyGain)
        if (individualEntrpyGain > maxInfoGain):
            maxInfoGain = individualEntrpyGain
            maxGainAttr = att;
    
    # If there's no gain at all, nothing has to be done, just return the original set
    if (maxInfoGain < 1e-6):
        #print("in maxInfoGain")
        #print(maxInfoGain,maxGainAttr)
        return None, None
    else:
        return maxGainAttr,maxInfoGain



In [6]:
def trueSet(s):
    return len(set(s)) == 1

def createTree_proto(df):
    
    x = df.drop(labels=targetAttribute, axis=1)
    y = df[targetAttribute]
    
    # If there could be no split, just return the original set
    if trueSet(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    
    maxGainAttr,maxInfoGain = calcInforGainForEachAtrribute(df)
    
    if((maxGainAttr== None) and (maxInfoGain == None)):
        return y

    # We split using the selected attribute
    sets = df.groupby(maxGainAttr)
    print(sets)
    for key,value in sets:
        print(key)
        
    print(type(sets))
    res = {}
    for k, v in sets:
        res["x_%s = %s" % (maxGainAttr, k)] = createTree(v)

    return res



In [21]:
def trueSet(s):
    return len(set(s)) == 1

def createTree(df):
    
    x = df.drop(labels=targetAttribute, axis=1)
    y = df[targetAttribute]
    
    # If there could be no split, just return the original set
    if trueSet(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    
    maxGainAttr,maxInfoGain = calcInforGainForEachAtrribute(df)
    
    if((maxGainAttr== None) and (maxInfoGain == None)):
        return y

    # We split using the selected attribute
    sets = df.groupby(maxGainAttr)
    #print(sets)
    
    #print(type(sets))
    res = {}
    for k, v in sets:
        res["x_%s = %s" % (maxGainAttr, k)] = createTree(v)

    return res


In [24]:
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'scipy'

In [22]:
def main():
    #df = pd.read_excel("MushroomDataSet_After_PreProcessing.xlsx")
    
    datasetAfterProcessing = pd.read_excel('MushroomDataSet_After_PreProcessing.xlsx')
    split_value = (int) (len(datasetAfterProcessing.index)*0.8)
    training_data = pd.DataFrame(datasetAfterProcessing.iloc[:split_value,:].values)
    training_data.columns= ['Poisonous/Edible','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring	','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color',	'population','habitat']
    
    targetValue = initializeList(training_data)
    
    
    print(createTree(training_data))
    
    '''
    tempDict = df.groupby("odor")
    for k,v in tempDict:
        print("key :",k)
        print("Length: ",len(v.groupby(targetAttribute)))
        print("-------------------")
    
    
    
    #temp = calcInforGainForEachAtrribute(df)
    #print(temp)
    #print(createTree(df).keys())

    
    df = df.drop(labels=targetAttribute, axis=1)
    print(df.head())
    
    print(allAttr.remove(targetAttribute))
    x = df[:,allAttr.remove(targetAttribute)]
    print(x)
    
    maxGainAttr,maxInfoGain = calcInforGainForEachAtrribute(df)
    
    if np.all(maxInfoGain < 1e-6):
        return y
    print (maxGain)
    '''
if __name__ == "__main__": main()

{'x_odor = a': 1       e
5       e
6       e
9       e
11      e
12      e
20      e
23      e
26      e
29      e
38      e
40      e
44      e
45      e
50      e
55      e
57      e
59      e
61      e
67      e
68      e
73      e
74      e
79      e
86      e
92      e
95      e
97      e
98      e
99      e
       ..
1338    e
1401    e
1463    e
1467    e
1501    e
1513    e
1514    e
1518    e
1528    e
1548    e
1557    e
1588    e
1662    e
1690    e
1724    e
1728    e
1751    e
1769    e
1773    e
1784    e
1813    e
1815    e
1856    e
1931    e
2017    e
2018    e
2045    e
2051    e
2059    e
2092    e
Name: Poisonous/Edible, Length: 400, dtype: object, 'x_odor = c': 2210    p
2285    p
2426    p
2482    p
2512    p
2540    p
2560    p
2627    p
2683    p
2749    p
2797    p
2841    p
2860    p
2887    p
2892    p
3004    p
3013    p
3026    p
3037    p
3047    p
3052    p
3057    p
3058    p
3061    p
3068    p
3095    p
3097    p
3104    p
3110    p
3118    p
       ..